# Spark

"A fast and general engine for large-scale data processing."

Distributes the work/operations on clusters (spread the work) - Resilient Distributed Dataset (RDD)

![](https://www.researchgate.net/publication/327926641/figure/fig2/AS:675666567647232@1538102871025/Execution-model-of-Spark.png)

Cluster manager: Spark

## RDD - Resilient Distributed Dataset

"A Resilient Distributed Dataset (RDD) is a read-only collection of data in Spark that can be partitioned across multiple machines in a cluster, allowing for parallel computation and fault tolerance through lineage reconstruction."

Transforming RDD's

- map

- flatmap

- filter

- distinct

- sample

- union, intersection, substract, cartesian

In [ ]:
# map example
rdd = sc.parallelize([1,2,3,4])
rdd.map(lambda x: x*x)

RDD Actions

- collect

- count

- countByValue (unique values)

- take

- top

- reduce (combining..)


Nothing actually happens in your driver program util action is called!


## MLLibs - Machine Learning libraries

MLLib Capabilites:

- Feature extraction: term frequency, inverse document frequency
- Basic statistics: chi-squared test, pearson or spearman correlation, min, max, mean, variance
- Linear regression, logistic regression
- Support Vector Machines (SVM)
- Naive Bayes classifier
- Decision trees
- K-Means clustering
- Principal component analysis, singular value decomposition (svd)
- Recommendations using Alternating Least Squares

MLLib Data types:

- Vector (dense or sparse - only store data that exists)
- LabeledPoint
- Rating

# Decision tree with MLLib

In [3]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=7948a993b9abdb866d18e715dfb0e26b37df4b8c5d4e989bf5c5ffca14f4761b
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


In [8]:
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.tree import DecisionTree
from pyspark import SparkConf, SparkContext
from numpy import array

In [6]:
conf = SparkConf().setMaster('local').setAppName('SparkDecisionTree')
sc = SparkContext(conf = conf)

In [9]:
def binary(YN):
  if (YN=='Y'):
    return 1
  else:
    return 0

def mapEducation(degree):
  if (degree == 'BS'):
    return 1
  elif (degree == 'MS'):
    return 2
  elif (degree == "PhD"):
    return 3
  else:
    return 0

In [14]:
def createLabeledPoints(fields):
  yearsExperience = int(fields[0])
  employed = binary(fields[1])
  previousEmployers = int(fields[2])
  educationLevel = mapEducation(fields[3])
  topTier = binary(fields[4])
  interned = binary(fields[5])
  hired = binary(fields[6])

  return LabeledPoint(hired, array([yearsExperience, employed, previousEmployers,
                                    educationLevel, topTier, interned]))

In [17]:
rawData = sc.textFile("PastHires.csv") #rdd
header = rawData.first()
rawData = rawData.filter(lambda x: x!= header) #filter out the column names

In [18]:
#split each line into a list based on the comma delimiters
csvData = rawData.map(lambda x: x.split(","))

In [19]:
# convert these lists to labeled points
trainingData = csvData.map(createLabeledPoints)

Creating a test candidate with 10 years of experience, currently employed, with a BS but from a non-top-tier school and did not do an internship

In [21]:
testCandidates = [array([10,1,3,1,0,0])]
testData = sc.parallelize(testCandidates)

In [23]:
model = DecisionTree.trainClassifier(trainingData, numClasses=2,
                                     categoricalFeaturesInfo={1:2,3:4,4:2,5:2},
                                     impurity='gini',
                                     maxDepth=5,
                                     maxBins=32)

In [24]:
predictions = model.predict(testData)
print('Hire prediction: ')
results = predictions.collect()
for result in results:
  print(result)

Hire prediction: 
1.0


In [25]:
print("Learned classification tree model:")
print(model.toDebugString())

Learned classification tree model:
DecisionTreeModel classifier of depth 4 with 9 nodes
  If (feature 1 in {0.0})
   If (feature 5 in {0.0})
    If (feature 0 <= 0.5)
     If (feature 3 in {1.0})
      Predict: 0.0
     Else (feature 3 not in {1.0})
      Predict: 1.0
    Else (feature 0 > 0.5)
     Predict: 0.0
   Else (feature 5 not in {0.0})
    Predict: 1.0
  Else (feature 1 not in {0.0})
   Predict: 1.0

